In [ ]:
!which python

In [ ]:
# interact -p GPU-shared --gres=gpu:v100-32:1 -t 1:00:00
# conda activate /ocean/projects/cis250085p/shared/envPreproces
# jupyter notebook --no-browser --ip=0.0.0.0

# module load  AI/pytorch_23.02-1.13.1-py3


In [ ]:
# module avail AI/pytorch_23.02-1.13.1-py3

# srun --jobid=32754720 --pty bash

# # !module load ffmpeg/4.3.1


# import os

# os.environ["PATH"] += os.pathsep + "/opt/packages/ffmpeg/4.3.1"

In [ ]:
processed_data = '/shared/B_track/processed/audio'
raw_data = '/shared/A_track/audio'

In [ ]:
import os

processed_data = '/shared/B_track/audio'
raw_data = '/shared/track_b_audio_files'

processed_files = os.listdir(processed_data)
processed_files = [f[:-7] for f in processed_files]
raw_files = os.listdir(raw_data)
print(f"Number of raw files: {len(raw_files)}")
print(f"Number of processed files: {len(processed_files)}")

In [ ]:
# processed_files = set(processed_files)
files_to_process = [f for f in raw_files if f not in processed_files]
print(f"Number of files to process: {len(files_to_process)}")

In [ ]:

# audio_path = "/shared/A_track/1736843853-lWPStQITn7XCnMqYWAPbr3y3blg1"

# processed_path = "/shared/A_track/processed/"


In [ ]:
import pandas as pd

train_json_path = "/shared/B_track/train.json"
train_df = pd.read_json(train_json_path).T
train_df.head()

In [ ]:
len(train_df)

In [ ]:
import torchaudio
import torch
import numpy as np
# from datasets import Dataset
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
raw_path = "/shared/A_track/"


def load_audio_and_compute_mel(example, target_sr=16000):
    path = example
    waveform, sr = torchaudio.load(raw_path + path)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr,)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)  # mono
    mel = feature_extractor(waveform.squeeze(), sampling_rate=target_sr
    , device="cuda"
    ).input_features[0]
    torch.save(mel, raw_path + "processed/{}.mel.pt".format(example))
    # return mel

for i in range(len(files_to_process)):
    load_audio_and_compute_mel("audio/" +files_to_process[i])
    if i % 100 ==0:
        print(i +1 , "/", len(files_to_process))

In [ ]:
import os

num_cpu_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cpu_cores}")

In [ ]:
# !module load ffmpeg/4.3.1


In [ ]:
import pandas as pd

train_json_path = "/shared/A_track/train.json"
train_df = pd.read_json(train_json_path).T
train_df.head()

In [ ]:
train_df.size

In [ ]:
df

In [ ]:

import IPython.display as ipd

audio_file = "output.wav"
ipd.Audio(audio_file)

In [ ]:
import os
import shutil
import json

# First, install the Kaggle API if not already installed
# !pip install kaggle

# Set up Kaggle API credentials (upload your kaggle.json or set environment variables)
# from google.colab import files
# files.upload()  # Upload kaggle.json


# os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
# os.environ['KAGGLE_KEY'] = 'your_kaggle_key'


# Zip the processed directory
shutil.make_archive('/A_track/processed', 'zip', '/A_track/processed')

# Create a dataset metadata file
dataset_metadata = {
    "title": "A Track Processed Audio",
    "id": "your-kaggle-username/a-track-processed-audio",
    "licenses": [{"name": "CC0-1.0"}],
    "private": True
}

with open('/A_track/processed/dataset-metadata.json', 'w') as f:
    json.dump(dataset_metadata, f, indent=4)

# Use Kaggle API to create the dataset
# !kaggle datasets create -p /shared/A_track/processed --dir-mode zip

# Note: Replace 'your-kaggle-username' with your actual Kaggle username.
# The dataset will be private by default due to "private": True in metadata.

In [ ]:
# Create a dataset metadata file
dataset_metadata = {
    "title": "A Track Processed Audio",
    "id": "mansourhassan/a-track-processed-audio",
    "licenses": [{"name": "CC0-1.0"}],
    "private": True
}

with open('/A_track/processed/dataset-metadata.json', 'w') as f:
    json.dump(dataset_metadata, f, indent=4)


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'key'
os.environ['KAGGLE_KEY'] = "key"


In [ ]:
!pip install kaggle

In [ ]:

!kaggle datasets create -p shared/A_track -q --dir-mode tar

In [ ]:
!kaggle datasets init -p /shared/A_track

In [ ]:
!nano /shared/A_track/processed/dataset-metadata.json

In [ ]:
!zip -0 -r processed_track_a.zip /shared/A_track/processed

In [ ]:
!zip -0 -r -P 16 processed_track_a_5_process.zip /shared/A_track/processed

In [ ]:
import time
from concurrent.futures import ProcessPoolExecutor

def sum_range(start, end):
    return sum(range(start, end))

def sum_serial():
    start_time = time.time()
    total = sum(range(0, 100001))
    elapsed = time.time() - start_time
    print(f"Serial sum: {total}, Time: {elapsed:.4f} seconds")
    return elapsed

def sum_parallel(num_workers=15):
    chunk_size = 100001 // num_workers
    ranges = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_workers)]
    ranges[-1] = (ranges[-1][0], 100001)  # Ensure last chunk goes to 100000

    start_time = time.time()
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(lambda args: sum_range(*args), ranges))
    total = sum(results)
    elapsed = time.time() - start_time
    print(f"Parallel sum: {total}, Time: {elapsed:.4f} seconds")
    return elapsed

serial_time = sum_serial()
parallel_time = sum_parallel()
print(f"Speedup: {serial_time/parallel_time:.2f}x")